<a href="https://colab.research.google.com/github/joaopedropastura/credit_score_ai/blob/main/v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn
!pip install pandas
!pip install joblib


In [2]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from joblib import load, dump
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.kernel_approximation import RBFSampler


In [94]:
path = "/content/credit_score.csv"

source = pd.read_csv(path)
le = LabelEncoder()

<ipython-input-94-0f7882e52b77>:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  source = pd.read_csv(path)


In [95]:
source = source[source['Payment_Behaviour'] != '!@9#%8']

In [96]:
sla = source['Type_of_Loan'].unique()
#   ||

# source = source[source['Type_of_Loan'] != 'Not Specified' ]

source = source[(source['Type_of_Loan'] != 'Not Specified') & (source['Type_of_Loan'].notnull())]

# source = source[source['Type_of_Loan'].dropna()]

In [97]:
def convert_to_months(age):
    if type(age) == type('String'):
      age_split = age.split(' ')
      years = int(age_split[0])
      months = int(age_split[3])
      # Cálculo total em meses
      total_months = years * 12 + months
      return total_months


source['Credit_History_Age'] = source['Credit_History_Age'].apply(convert_to_months)


In [98]:
source = source.drop(['ID', 'Customer_ID', 'SSN', 'Name'], axis=1)

In [60]:
source = source.head(3000)


In [99]:
Month = source["Month"].unique()
Credit_History_Age = source['Credit_History_Age'].unique()
Occupation = source["Occupation"].unique()
Credit_Mix = source["Credit_Mix"].unique()
Credit_Score = source["Credit_Score"].unique()
Payment_Behaviour = source['Payment_Behaviour'].unique()
Payment_of_Min_Amount = source["Payment_of_Min_Amount"].unique()
Type_of_Loan = source["Type_of_Loan"].unique()

In [100]:
le.fit(Month)
source['Month'] = le.transform(source['Month'])

le.fit(Type_of_Loan)
source['Type_of_Loan'] = le.transform(source['Type_of_Loan'])

le.fit(Payment_of_Min_Amount)
source['Payment_of_Min_Amount'] = le.transform(source['Payment_of_Min_Amount'])

le.fit(Credit_History_Age)
source['Credit_History_Age'] = le.transform(source['Credit_History_Age'])

le.fit(Occupation)
source['Occupation'] = le.transform(source['Occupation'])

le.fit(Credit_Mix)
source['Credit_Mix'] = le.transform(source['Credit_Mix'])

le.fit(Credit_Score)
source['Credit_Score'] = le.transform(source['Credit_Score'])

le.fit(Payment_Behaviour)
source['Payment_Behaviour'] = le.transform(source['Payment_Behaviour'])

In [101]:
source["Annual_Income"] = source["Annual_Income"].str[:-1]

In [102]:
def filterAge(age):
  try:
    pattern = r'\b\d+(?:[-_]\d+)*\b\s+'
    number = re.findall(pattern, age)
    return ' '.join(number)
  except:
    return age


for elem in source:

  source[elem] = source[elem].apply(filterAge)



In [103]:
for elem in source:
  source[elem].replace('', 0, inplace=True)

source = source.fillna(0)

In [ ]:
# for column in source:
#   source[column] = source[column] /source[column].abs().max()



# source
# source = source.fillna(0)

In [104]:
source


,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,3,0,12,0,1824.843333,3,4,3,0,128,...,3,0,26.822620,264,1,49.574949,0.0,2,0.000000,0
1,2,0,12,0,0.000000,3,4,3,0,128,...,1,0,31.944960,404,1,49.574949,0.0,3,0.000000,0
2,6,0,12,0,0.000000,3,4,3,0,128,...,1,0,28.609352,266,1,49.574949,0.0,4,0.000000,0
3,0,0,12,0,0.000000,3,4,3,0,128,...,1,0,31.377862,267,1,49.574949,0.0,5,0.000000,0
4,7,0,12,0,1824.843333,3,4,3,0,128,...,1,0,24.797347,268,1,49.574949,0.0,1,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,6,0,9,0,3359.415833,4,6,7,0,683,...,3,0,39.323569,376,1,35.104023,0.0,1,410.256158,1
99995,0,0,9,0,3359.415833,4,6,7,0,683,...,3,0,34.663572,377,1,35.104023,0.0,0,479.866228,1
99996,7,0,9,0,3359.415833,4,6,7,0,683,...,3,0,40.565631,378,1,35.104023,0.0,1,496.651610,1
99997,5,0,9,0,3359.415833,4,6,5729,0,683,...,1,0,41.255522,379,1,35.104023,0.0,0,516.809083,1


In [105]:
X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

X_train, X_test, Y_train, Y_test = \
  train_test_split(X, Y, test_size=0.2, random_state=50)

In [109]:
#RANDOM FOREST COM PCA

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = DecisionTreeClassifier(max_depth=100, min_samples_split=36)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


0.6416604616530156
0.8122401489295688


In [ ]:
#RANDOM FOREST COM PCA


X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = RandomForestClassifier(max_depth=100, min_samples_split=5)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


In [108]:
from sklearn.ensemble import GradientBoostingClassifier

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Definição do modelo Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=500)

# Treinando o modelo
gb_model.fit(X_train, Y_train)

# Fazendo previsões no conjunto de teste
Y_pred_gb = gb_model.predict(X_test)

# Calculando a precisão do modelo Gradient Boosting no conjunto de teste
accuracy_gb = accuracy_score(Y_test, Y_pred_gb)
print("Precisão do Gradient Boosting Classifier no conjunto de teste:", accuracy_gb)

# Matriz de Confusão
matrix_gb = confusion_matrix(Y_test, Y_pred_gb)
print("Matriz de Confusão do Gradient Boosting Classifier:\n", matrix_gb)


Precisão do Gradient Boosting Classifier no conjunto de teste: 0.6698932737652022
Matriz de Confusão do Gradient Boosting Classifier:
 [[1602   96  952]
 [ 273 3291 1407]
 [ 944 1648 5903]]


In [75]:
#melhores parametros para teste

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix



# Seu código para divisão de dados e aplicação do PCA (como você já fez)
X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


# Definição do modelo RandomForestClassifier
model = RandomForestClassifier()

# Parâmetros que serão testados na busca em grade
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 50, 100],
    'min_samples_split': [2, 5, 10]
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Treinando o modelo com a busca em grade
grid_search.fit(X_train, Y_train)

# Obtendo os melhores parâmetros encontrados
best_params = grid_search.best_params_
print("Melhores parâmetros:", best_params)

# Avaliando o modelo com os melhores parâmetros no conjunto de teste
best_model = grid_search.best_estimator_
Y_pred = best_model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("Precisão no conjunto de teste com os melhores parâmetros:", accuracy)

# Se desejar, você pode também analisar a matriz de confusão ou outras métricas para avaliar o modelo final
matrix = confusion_matrix(Y_test, Y_pred)
print("Matriz de Confusão:\n", matrix)


Melhores parâmetros: {'max_depth': 100, 'min_samples_split': 2, 'n_estimators': 200}
Precisão no conjunto de teste com os melhores parâmetros: 0.7566666666666667
Matriz de Confusão:
 [[ 58   3  33]
 [  3 137  23]
 [ 26  58 259]]


In [70]:
#SVM COM PCA

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


model = SVC()

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


0.57
0.5208333333333334


In [71]:
#NUSVC COM PCA

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


model = make_pipeline(StandardScaler(), NuSVC())

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


ValueError: ignored

In [72]:
#SGD

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)

0.2916666666666667
0.3129166666666667


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [73]:
#Kernel Approximation


X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X_train)

model = SGDClassifier(max_iter=5)


model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = model.score(Y_test, Y_pred)
accurac3y = model.score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


ValueError: ignored